Licensed to the Apache Software Foundation (ASF) under one or more contributor license agreements. See the NOTICE file distributed with this work for additional information regarding copyright ownership. The ASF licenses this file to you under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [0]:
import datetime
import theano
import theano.tensor as tt
import pandas as pd
import numpy as np
import pymc3 as pm

#theano.config.gcc.cxxflags="-Wno-c++11-narrowing"
theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"

#ADD THIS LOGIC TO THE APPLY_DELAY FUNCTION DIRECTLY
#def lognormal_tensor(x, mean, sigma):
#  dist = tt.exp(-((tt.log(x)-mean) **2)/ (2*sigma**2)
#  return dist/tt.sum(dist, axis=0)
def make_delay_matrix(n_rows, n_columns, initial_delay=0):
    """
        Has in each entry the delay between the input with size n_rows and the output
        with size n_columns

        initial_delay is the top-left element.
    """
    size = max(n_rows, n_columns)
    mat = np.zeros((size, size))
    for i in range(size):
        diagonal = np.ones(size - i) * (initial_delay + i)
        mat += np.diag(diagonal, i)
    for i in range(1, size):
        diagonal = np.ones(size - i) * (initial_delay - i)
        mat += np.diag(diagonal, -i)
    return mat[:n_rows, :n_columns]

def delay_cases(new_I_t, len_new_I_t, len_out, delay, delay_diff):
    """
        Delays (time shifts) the input new_I_t by delay.

        Parameters
        ----------
        new_I_t : ~numpy.ndarray or theano vector
            Input to be delayed.

        len_new_I_t : integer
            Length of new_I_t. (Think len(new_I_t) ).
            Assure len_new_I_t is larger then len(cum_confirmed_positive)-delay, otherwise it
            means that the simulated data is not long enough to be fitted to the data.

        len_out : integer
            Length of the output.

        delay : number
            If delay is an integer, the array will be exactly shifted. Else, the data
            will be shifted and intepolated (convolved with hat function of width one).
            Take care that delay is smaller than or equal to delay_diff,
            otherwise zeros are returned, which could potentially lead to errors.

        delay_diff: integer
            The difference in length between the new_I_t and the output.

        Returns
        -------
            an array with length len_out that was time-shifted by delay
    """

    # elementwise delay of input to output
    delay_mat = make_delay_matrix(
        n_rows=len_new_I_t, n_columns=len_out, initial_delay=delay_diff
    )
    inferred_cases = itplt(new_I_t, delay, delay_mat)
    return inferred_cases

def delay(rows, columns, delay=0): #make_Delay_matrix
  #This looks at the shape of the parameters, and the delay, in order to create
  #a delay matrix with numbers starting from the diagonal (the diagonal takes the value of the delay and the next values, 
  #follow an arithmetic progression with a unit increase)
  size= max(rows, columns)
  out = np.zeros((size, size))
  for i in range(size):
    del_new= np.ones(size-i)*(delay+i)
    out = out + np.diag(del_new, i)
  for i in range(1, size):
    del_new = np.ones(size-i)*(delay-i)
  return out[:rows, :columns]
#Function lognormal_tensor added to the function delayed

def itplt(arr, delay, datadelay): #Data smoothing function
  itplt_data = tt.maximum(1 - tt.abs_(datadelay - delay), 0)
  dotprod = tt.dot(arr, itplt_data)
  return dotprod

def delay_cases_lognormal(
    input_arr,
    len_input_arr,
    len_output_arr,
    median_delay,
    scale_delay,
    delay_t,
):
    delay_mat = delay(
        rows=len_input_arr,
        columns=len_output_arr,
        delay=delay_t,
    )
    delay_mat[
        delay_mat < 0.01
    ] = 0.01  # needed because negative values lead to nans in the lognormal distribution.
    delayed_arr = delayed(input_arr, median_delay, scale_delay, delay_mat)
    return delayed_arr

def delayed(arr, delay, delay_shape, datadelay): #apply_delay and tt_lognormal
  distribution = tt.exp(-((tt.log(datadelay)-np.log(delay))**2)/ (2*delay_shape **2))
  arr2 = distribution/tt.sum(distribution, axis=0)
  return tt.dot(arr, arr2)

def infer_delayed(I_rate_t,I_tperiod,  output_length, delay, diff_I_output):
  delayed_initial = delay(rows= I_tperiod, columns = output_length, delay = diff_I_output)
  delay_inferred = itplt(I_rate_t, delay, delayed_initial)
  return delay_inferred

def dist_smooth(v1, vk, t1, tk, t_total):
  t = np.arange(t_total)
  smooth = tt.clip((t - t1)/(tk- t1), 0,1) * (vk - v1) + v1 #Smoothing with delta values
  return smooth 
#output=delay(rows= 6,columns = 4, delay = 2)
#print(output)
def SIR_MOD(daily_positive_cases, ordered_list_of_gov_interventions, date1, constant_parameters ):

  
  #Adding default values for when shapes and parameters are not defined in the change point list
  #for key, value in prior_information_0.items():
  #  if key not in prior_information:
  #    prior_information[key] = value
  

  svi_mnrty_wt_avg = 0.8903 #CA:0.8903 #NY: 0.479 #NJ:0.66254
  with pm.Model() as sim: 
    I_start = pm.HalfNormal(name = 'start_inf', sigma = constant_parameters['prior_beta_I_start']/(1+svi_mnrty_wt_avg))


    list_infections = []

    for i, sd_pt in enumerate(ordered_list_of_gov_interventions):
      list_infections.append(
          pm.Lognormal(
              name=f'Inf_rate_{i}',
              mu=np.log(sd_pt['prior_inf_rate_median'] ),
              sigma = sd_pt['prior_inf_rate_sigma']
          )
      )
          
    cp_transient_list = []
    prev_date = date1
    for i, sd_pt in enumerate(ordered_list_of_gov_interventions[1:]):
        
        transient_start = sd_pt['prior_mean_transient']
        prior_mean = (transient_start - prev_date).days
      
        tr_start = pm.Normal(
                name=f"transient_start_{i}",
                mu=prior_mean,
                sigma=sd_pt["prior_variance_date_start_transient"],
            )
        cp_transient_list.append(tr_start)
        dt_before = transient_start
        # same for transient times
    tr_len_list = []
    for i, cp in enumerate(ordered_list_of_gov_interventions[1:]):
      tr_len = pm.Lognormal(
          name=f"transient_len_{i}",
          mu=np.log(cp["prior_median_transient_len"]),
          sigma=cp["prior_variance_transient_len"],)
      tr_len_list.append(tr_len)

    Inf_rate_t_list = [list_infections[0] * tt.ones(constant_parameters['num_days_simulation'])]
    Inf_rate_before = list_infections[0]

    for tr_start, tr_len, Inf_rate_after in zip(
            cp_transient_list, tr_len_list, list_infections[1:]
        ):
        Inf_rate_t = dist_smooth(
                          v1=0,
                vk=1,
                t1=tr_start,
                tk=tr_start + tr_len,
                t_total=constant_parameters['num_days_simulation'],
            ) * (Inf_rate_after - Inf_rate_before)
        Inf_rate_before = Inf_rate_after
        Inf_rate_t_list.append(Inf_rate_t)
    Inf_rate_t = sum(Inf_rate_t_list)

        # fraction of people that recover each day, recovery rate mu
    mu = pm.Lognormal(
            name="mu",
            mu=np.log(constant_parameters["prior_median_mu"]),
            sigma=constant_parameters["prior_variance_mu"],
        )

        # delay in days between contracting the disease and being recorded
    delay = pm.Lognormal(
            name="delay",
            mu=np.log(constant_parameters["prior_median_delay"]),
            sigma=constant_parameters["prior_variance_delay"],
        )

        # prior of the error of observed cases
    sigma_obs = pm.HalfNormal("sigma_obs", sigma=constant_parameters["prior_beta_variance_obs"])

        # -------------------------------------------------------------------------- #
        # training the model with loaded data provided as argument
        # -------------------------------------------------------------------------- #

    S_start = constant_parameters['tot_pop'] - I_start

    new_I_0 = tt.zeros_like(I_start)
    S, I, new_I = Model_simulation(
            Inf_rate_t=Inf_rate_t, mu=mu, S_start=S_start, I_start=I_start, N=constant_parameters['tot_pop']
        )
    '''
    def subsequent_day_parameters(Inf_rate_t, S_t, I_t, _,mu, tot_pop):
      new_I_t = Inf_rate_t / tot_pop * I_t * S_t
      S_t = S_t - new_I_t
      I_t = I_t + new_I_t - mu * I_t
      I_t = tt.clip(I_t, 0, tot_pop) #for stability
      return S_t, I_t, new_I_t

      # first tuple of theano scan will return S, I, new_I
    outputs, _ = theano.scan(
        fn=subsequent_day_parameters,
        sequences=[Inf_rate_t],
        outputs_info=[S_start, I_start, new_I_0],
        non_sequences=[mu, constant_parameters['tot_pop']],)

    S, I, new_I = outputs
    '''

    new_cases_inferred = delay_cases(
        new_I_t=new_I,
        len_new_I_t=constant_parameters['num_days_simulation'],
        len_out=constant_parameters['num_days_simulation'] - constant_parameters['diff_data_simulation'],
        delay=delay,
        delay_diff=constant_parameters['diff_data_simulation'],)
    
    
    new_cases_inferred_eff = new_cases_inferred
    num_days_data = daily_positive_cases.shape[-1]
    
    pm.StudentT(
        name="_new_cases_studentT",
        nu=4,
        mu=new_cases_inferred_eff[:num_days_data],
        sigma=tt.abs_(new_cases_inferred[:num_days_data] + 1) ** 0.5
        * sigma_obs,  #+1 and tt.abs to avoid nans
        observed=daily_positive_cases,)
    
    pm.Deterministic("Inf_rate_t", Inf_rate_t)
    pm.Deterministic("new_cases", new_cases_inferred_eff)
    pm.Deterministic("new_cases_raw", new_cases_inferred)

  return sim

def Model_simulation(Inf_rate_t, mu, S_start, I_start, N):
    """
        Implements the susceptible-infected-recovered model

        Parameters
        ----------
        Inf_rate_t : ~numpy.ndarray
            time series of spreading rate, the length of the array sets the
            number of steps to run the model for

        mu : number
            recovery rate

        S_start : number
            initial number of susceptible at first time step

        I_start : number
            initial number of infected

        N : number
            population size

        Returns
        -------
        S : array
            time series of the susceptible

        I : array
            time series of the infected

        new_I : array
            time series of the new infected
    """

    new_I_0 = tt.zeros_like(I_start)

    def next_day(Inf_rate_t, S_t, I_t, _, mu, N):
        new_I_t = Inf_rate_t / N * I_t * S_t
        S_t = S_t - new_I_t
        I_t = I_t + new_I_t - mu * I_t
        I_t = tt.clip(I_t, 0, N)  # for stability
        return S_t, I_t, new_I_t

    # theano scan returns two tuples, first one containing a time series of
    # what we give in outputs_info : S, I, new_I
    outputs, _ = theano.scan(
        fn=next_day,
        sequences=[Inf_rate_t],
        outputs_info=[S_start, I_start, new_I_0],
        non_sequences=[mu, N],
    )

    return outputs

In [12]:
import os
import pandas as pd
os.getcwd()

'/Users/amanchawla'

In [13]:
data=pd.read_csv('Confirmed_cases_by_state.csv') #FOR STATE DATA
#data=pd.read_csv('C:\\Users\\Aman\\Documents\\covid_confirmed_usafacts_1604.csv') #FOR COUNTY DATA

population_lockdown_dates=pd.read_csv('LockdownDates_populationMerged.csv')
country = 'United States'
state_codes=data.columns[1:]

date_data_begin = max(pd.to_datetime(data['Dates'], infer_datetime_format=True)).to_pydatetime() - datetime.timedelta(days=40)
date_data_end   = max(pd.to_datetime(data['Dates'], infer_datetime_format=True)).to_pydatetime()
data2=data[pd.to_datetime(data['Dates'], infer_datetime_format=True)>=date_data_begin]


population_lockdown_dates.loc[population_lockdown_dates['lockdown_date']=='YTA','lockdown_date']=date_data_end
population_lockdown_dates.loc[population_lockdown_dates['partial_date']=='YTA','lockdown_date']=date_data_end
population_lockdown_dates.loc[population_lockdown_dates['partial_date']=='YTA','partial_date']=date_data_end

population_lockdown_dates['partial_date']=population_lockdown_dates['partial_date'].fillna(date_data_end)
population_lockdown_dates['lockdown_date']=population_lockdown_dates['lockdown_date'].fillna(date_data_end)
population_lockdown_dates['lockdown_date']=pd.to_datetime(population_lockdown_dates['lockdown_date'], infer_datetime_format=True)
population_lockdown_dates.loc[population_lockdown_dates['lockdown_date']<date_data_begin,'lockdown_date']=date_data_end
population_lockdown_dates['partial_date']=pd.to_datetime(population_lockdown_dates['partial_date'], infer_datetime_format=True)
population_lockdown_dates.loc[population_lockdown_dates['lockdown_date']<population_lockdown_dates['partial_date'],'partial_date']=population_lockdown_dates['lockdown_date']
population_lockdown_dates.dtypes

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(data.head())


#FOR STATE DATA (i should correspond to the state code we're running it for in state_code)
i=4 #4: California; 31: NEw Jersey, 34: New York
state=state_codes[i]
cases_obs = data2[state].values

population_lockdown_dates_sub=population_lockdown_dates.loc[population_lockdown_dates['State Code']==state]
population_lockdown_dates_sub

#prior_date_mild_dist_begin = datetime.datetime.utcfromtimestamp(population_lockdown_dates_sub['partial_date'].values[0].tolist()/1e9)- datetime.timedelta(days = 3) 
#prior_date_mild_dist_begin

diff_data_sim = 14
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim) 

prior_date_mild_dist_begin = datetime.datetime.utcfromtimestamp(population_lockdown_dates_sub['partial_date'].values[0].tolist()/1e9)- datetime.timedelta(days = 3) 
prior_date_strong_dist_begin =  datetime.datetime.utcfromtimestamp(population_lockdown_dates_sub['partial_date'].values[0].tolist()/1e9)- datetime.timedelta(days = 2) 
prior_date_contact_ban_begin =  datetime.datetime.utcfromtimestamp(population_lockdown_dates_sub['lockdown_date'].values[0].tolist()/1e9)- datetime.timedelta(days = 1) 

num_days_data = (date_data_end-date_data_begin).days
num_days_future = 110
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_data_end   + datetime.timedelta(days = num_days_future)
num_days_sim = (date_end_sim-date_begin_sim).days


cases_obs = data2[state].values

#what if scenarios
#intervention attribute dictionary


###################################################################
#dictionary format for adding new goverment interventions##########
###################################################################


intervetion_attributes = dict(
    #date of intervention begin
    prior_mean_transient=None,
    prior_variance_date_start_transient=None,
    
    #time for intervebtion to take effect
    prior_median_transient_len=None,
    prior_variance_transient_len=None,
    
    #infection rate
    prior_inf_rate_median= None,
    prior_inf_rate_sigma= None,
    
)

###################################################################
#define new goverment interventions################################
###################################################################

#defining default prior
default_prior = {
    #date of intervention begin
    'prior_mean_transient':None,
    'prior_variance_date_start_transient':None,                
    #time for intervebtion to take effect
    'prior_median_transient_len':None,
    'prior_variance_transient_len':None,
    #infection rate
    'prior_inf_rate_median': 0.4,
    'prior_inf_rate_sigma': 0.9}
                    
#defining changept 2 -  mild_social_distancing
mild_social_distancing = {
                    #date of intervention begin
                    'prior_mean_transient':prior_date_mild_dist_begin,
                    'prior_variance_date_start_transient':3,
                    
                    #time for intervebtion to take effect
                    'prior_median_transient_len':3,
                    'prior_variance_transient_len':0.3,
                    
                    #infection rate
                    'prior_inf_rate_median': 0.2,
                    'prior_inf_rate_sigma': 0.5
                    
                }

#defining changept 3 -  strong social distancing
strong_social_distancing = {
                    #date of intervention begin
                    'prior_mean_transient':prior_date_strong_dist_begin,
                    'prior_variance_date_start_transient':1,
                    
                    #time for intervebtion to take effect
                    'prior_median_transient_len':3,
                    'prior_variance_transient_len':0.3,
                    
                    #infection rate
                    'prior_inf_rate_median': 1/8,
                    'prior_inf_rate_sigma': 0.5
                    
                }


#defining changept 3 -  strong social distancing
lock_down = {
                    #date of intervention begin
                    'prior_mean_transient':prior_date_contact_ban_begin,
                    'prior_variance_date_start_transient':1,
                    
                    #time for intervebtion to take effect
                    'prior_median_transient_len':3,
                    'prior_variance_transient_len':0.3,
                    
                    #infection rate
                    'prior_inf_rate_median': 1/8/2,
                    'prior_inf_rate_sigma': 0.5
                    
                }


ordered_list_of_gov_interventions = [ default_prior, mild_social_distancing, strong_social_distancing, lock_down]


constant_parameters = {
          'tot_pop': population_lockdown_dates_sub['Population'].values[0],
          'prior_beta_I_start' : 100,
          'prior_median_mu' : 0.12, 
          'prior_variance_mu' : 0.2,
          'prior_median_delay' : 5, 
          'prior_variance_delay' : 0.2,
          'prior_beta_variance_obs' : 10,
          #simulation inofrmation
          'num_days_simulation' : num_days_sim,
          'diff_data_simulation' : diff_data_sim
          }

traces = []
models=[]
for scenarios in np.arange(1,4):
  model = SIR_MOD(daily_positive_cases= np.diff(cases_obs), 
                ordered_list_of_gov_interventions = ordered_list_of_gov_interventions[:scenarios], 
                date1 = date_begin_sim, 
                constant_parameters = constant_parameters)
  models.append(model)
  traces.append(pm.sample(model=model, init='advi', draws=4000, tune=1000 ))


     Dates  AK  AL  AR  AZ  CA  CO  CT  DC  DE  FL  GA  HI  IA  ID  IL  IN  \
0  1/22/20   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1  1/23/20   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
2  1/24/20   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   
3  1/25/20   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   
4  1/26/20   0   0   0   1   2   0   0   0   0   0   0   0   0   0   1   0   

   KS  KY  LA  MA  MD  ME  MI  MN  MO  MS  MT  NC  ND  NE  NH  NJ  NM  NV  NY  \
0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

   OH  OK  OR  PA  RI  SC  SD  TN  TX  UT  V

Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average Loss = 304.08:   7%|▋         | 13691/200000 [02:03<28:07, 110.43it/s]
Convergence achieved at 13700
Interrupted at 13,699 [6%]: Average Loss = 407.92
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_obs, delay, mu, Inf_rate_0, start_inf]
Sampling 2 chains, 0 divergences: 100%|██████████| 10000/10000 [46:42<00:00,  3.57draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average Loss = 292.88:   8%|▊         | 15893/200000 [02:30<29:04, 105.52it/s]
Convergence achieved at 15900
Interrupted at 15,899 [7%]: Average Loss = 385.81
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_obs, delay, mu, transient_len_0, transient_start_0, Inf_rate_1, Inf_rate_0, start_inf]
Sampling 2 chains, 0 divergences: 100%|██████████| 10000/10000 [2:18:45<00:00,  1.20draws/s]
Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average Loss = 292.89:   8%|▊         | 16489/200000 [02:36<29:06, 105.

In [0]:
import matplotlib

In [31]:
def truncate_number(number, precision):
    return '{{:.{}f}}'.format(precision).format(number)  

def print_median_CI(arr, prec = 2):
    f_trunc = lambda n: truncate_number(n, prec)
    med = f_trunc(np.median(arr))
    perc1, perc2 = f_trunc(np.percentile(arr, q=2.5)), f_trunc(np.percentile(arr, q=97.5))
    return 'Median: {}\n95% CI: [{}, {}]'.format(med, perc1, perc2)
def conv_time_to_mpl_dates(arr):
    return matplotlib.dates.date2num([datetime.timedelta(days=float(date)) + date_begin_sim for date in arr])
traces_copy=traces
trace = traces[0]
posterior = traces

colors  = ['tab:green']
points = [ 'Mild Social Distancing', 'Strong Social Distancing','Total Lockdown']

new_cases_past=pd.DataFrame()
new_cases_Future=pd.DataFrame()
new_cases_Future_percentiles=pd.DataFrame()
new_cases_Future_percentiles_DF=pd.DataFrame()
for trace_scen, color, point in zip(posterior, colors, points):
    new_cases_past1 = trace_scen.new_cases[:,:num_days_data]
    new_cases_past[point]=np.median(new_cases_past1, axis=0)
    time2 = np.arange(0, num_days_future+1)
    mpl_dates_fut = conv_time_to_mpl_dates(time2) + diff_data_sim + num_days_data
    end_date = mpl_dates_fut[-10]
    cases_future = trace_scen['new_cases'][:, num_days_data:].T
    new_cases_Future[point] = np.median(cases_future, axis=-1)
    new_cases_Future_percentiles[point] = (
    np.percentile(cases_future, q=2.5, axis=-1),
    np.percentile(cases_future, q=97.5, axis=-1),
    )
   
    new_cases_Future_percentiles_DF[f'{point} - Lower Bound'] = new_cases_Future_percentiles[point][0]
    new_cases_Future_percentiles_DF[f'{point} - Upper Bound'] = new_cases_Future_percentiles[point][1]
    
Date_sim=[]
for i in range(1, len(time2)):
  Date_sim.append(date_begin_sim +datetime.timedelta(days=float(time2[i]))+datetime.timedelta(days=float(54)))
date_sim2=[]
for date in Date_sim:
  date_sim2.append(datetime.datetime.strftime(date, '%d/%m/%y'))
print(len(date_sim2[1:]))
date_sim3=date_sim2[1:]
new_cases_Future
new_cases_Future_cum=new_cases_Future.cumsum(axis=0)
new_cases_Future_cum1=new_cases_Future_cum+cases_obs[-1]
new_cases_Future_cum1

new_cases_Future_cum2=new_cases_Future_cum1
new_cases_Future_cum2.index=date_sim2
new_cases_Future_cum2 #SAVE THIS 
CUM_Cases_Future_percentiles=pd.DataFrame()
Cumulative_Cases=pd.DataFrame()
cum_cases_Future_percentiles_DF=pd.DataFrame()
for trace_scen,  point in zip(posterior, points[1:]):
    new_cases_past = trace_scen.new_cases[:,:num_days_data]
    cum_cases = np.cumsum(new_cases_past, axis=1) + cases_obs[0]
    Cumulative_Cases[point]=np.median(cum_cases, axis=0)
    time2 = np.arange(0, num_days_future+1)
    mpl_dates_fut = conv_time_to_mpl_dates(time2) + diff_data_sim + num_days_data
    cases_future = np.cumsum(trace_scen['new_cases'][:, num_days_data:].T, axis=0) + cases_obs[-1]
    
    #cases_future = np.concatenate([np.ones((1,cases_future.shape[1]))*cases_obs[-1], cases_future], axis=0)
    #Cumulative_Cases[legend] = np.median(cases_future, axis=-1)
    CUM_Cases_Future_percentiles[legend] = (
        np.percentile(cases_future, q=2.5, axis=-1),
        np.percentile(cases_future, q=97.5, axis=-1),)
    cum_cases_Future_percentiles_DF[f'{point}- Lower Bound'] = CUM_Cases_Future_percentiles[point][0]
    cum_cases_Future_percentiles_DF[f'{point} - Upper Bound'] = CUM_Cases_Future_percentiles[point][1]


cum_cases_Future_percentiles_DF.index=date_sim2
cum_cases_Future_percentiles_DF
cum_cases_Future_percentiles_DF['Date']=cum_cases_Future_percentiles_DF.index.astype('str')
new_cases_Future_cum2['Date']=new_cases_Future_cum2.index.astype('str')
FINAL_DF=pd.merge(cum_cases_Future_percentiles_DF, new_cases_Future_cum2, on="Date")


FINAL_DF.index=FINAL_DF['Date']
FINAL_DF2=FINAL_DF.copy()

#FINAL_DF2.to_csv('California_0804_Projection_Data2_1.csv')
#FINAL_DF2.to_csv('California_2604_Projection_Data2_1.csv')
#FINAL_DF2.to_csv('N_1604_Projection_Data2_1.csv')
FINAL_DF2.head()
FINAL_DF2.to_csv('CA_new_code_projection_08_04_3scenarios.csv')

109


KeyError: ignored

IndexError: ignored

In [0]:
def delay(rows, columns, delay=0): #make_Delay_matrix
  #This looks at the shape of the parameters, and the delay, in order to create
  #a delay matrix with numbers starting from the diagonal (the diagonal takes the value of the delay and the next values, 
  #follow an arithmetic progression with a unit increase)
  size= max(rows, columns)
  out = np.zeros((size, size))
  for i in range(size):
    del_new= np.ones(size-i)*(delay+i)
    out = out + np.diag(del_new, i)
  for i in range(1, size):
    del_new = np.ones(size-i)*(delay-i)
  return out[:rows, :columns]

In [7]:
delay(50,50,14)

array([[14., 15., 16., ..., 61., 62., 63.],
       [ 0., 14., 15., ..., 60., 61., 62.],
       [ 0.,  0., 14., ..., 59., 60., 61.],
       ...,
       [ 0.,  0.,  0., ..., 14., 15., 16.],
       [ 0.,  0.,  0., ...,  0., 14., 15.],
       [ 0.,  0.,  0., ...,  0.,  0., 14.]])

In [6]:
range(1,4)

range(1, 4)

In [7]:
print(range(1,4))

range(1, 4)


In [10]:
np.arange(1,4)

array([1, 2, 3])

In [9]:
a

range(1, 4)

Authors: Sachin C S (Cognizant), Aman Chawla (Cognizant)

Copyright {2020} Cognizant Technology Solutions
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.